# Dependencies 

In [1]:
!pip install -U onnx
!pip install -U onnxruntime
!pip install -U onnx-tf
!pip install -U tensorflow-addons
!pip install -U tensorflowjs
!pip install -U simple-onnx-processing-tools
!pip install -U nvidia-pyindex
!pip install -U onnx-graphsurgeon
!pip install -U tensorflow-probability
!pip install -U pyarrow
!pip install -U onnx_graphsurgeon
!pip install seaborn
!pip install --upgrade torch torchvision
!pip install ultralytics


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from onnx_tf.backend import prepare
from pathlib import Path
import onnx
from scc4onnx import order_conversion
from onnx_tf.backend import prepare
import tarfile


2023-07-12 16:52:47.327630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alejandrosalinas/miniconda3/envs/py38/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
import torch
print(torch.__version__)


2.0.1


# ONNX export 

In [4]:
%cd yoloDEP

/Users/alejandrosalinas/Desktop/Enhancing-Hospital-Efficiency-Through-Web-Deployed-Object-Detection/yoloDEP


In [5]:
!python export.py --weights runs/train/yolov8_hospital/weights/best.pt  \
        --grid  \
        --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 \
        --img-size 640 640 --max-wh 640 # For onnxruntime, you need to specify this value as an integer, when it is 0 it means agnostic NMS,
                     # otherwise it is non-agnostic NMS

Namespace(batch_size=1, conf_thres=0.35, device='cpu', dynamic=False, dynamic_batch=False, end2end=False, fp16=False, grid=True, img_size=[640, 640], include_nms=False, int8=False, iou_thres=0.65, max_wh=640, simplify=False, topk_all=100, weights='runs/train/yolov8_hospital/weights/best.pt')
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1 CPU

Model summary (fused): 168 layers, 3007208 parameters, 0 gradients

Starting TorchScript export with torch 2.0.1...
/Users/alejandrosalinas/miniconda3/envs/py38/lib/python3.8/site-packages/ultralytics/nn/modules/head.py:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif self.dynamic or self.shape != shape:
TorchScript export success, saved as runs/train/yolov8_hospital/weights/best.torchscript.pt
CoreML export failure: No module named 

# Prepare ONNX model for Tensorflow Backend

In [6]:
onnx_model_path = 'runs/train/yolov8_hospital/weights/best.onnx'
onnx_model = onnx.load(onnx_model_path)

In [9]:
tf_rep = prepare(onnx_model)
!mkdir ./runs/train/yolov8_hospital/weights/tf
!mkdir ./runs/train/yolov8_hospital/weights/tf/stockout_web_model

In [10]:
tf_model_dir = "./runs/train/yolov8_hospital/weights/tf"
tf_rep.export_graph(tf_model_dir)
tfjs_model_dir = f"{tf_model_dir}/hospital_web_model"

INFO:tensorflow:Assets written to: ./runs/train/yolov8_hospital/weights/tf/assets


INFO:tensorflow:Assets written to: ./runs/train/yolov8_hospital/weights/tf/assets


# Convert to tfjs

In [11]:
#Export SavedModel to TFJS format
tfjs_convert_command = f"""tensorflowjs_converter
                 --input_format=tf_saved_model
                 --output_format=tfjs_graph_model
                 --signature_name=serving_default
                 --saved_model_tags=serve
                 "{tf_model_dir}"
                 "{tfjs_model_dir}"
                 """
tfjs_convert_command = " ".join(tfjs_convert_command.split())

In [12]:
%sx $tfjs_convert_command

['weight StatefulPartitionedCall/zeros_22 with shape () and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/Const_82 with shape () and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/Shape_27 with shape (3,) and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/ExpandDims_17 with shape (1, 1) and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/Cast_71 with shape (1,) and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/Const_81 with shape () and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/ones_17 with shape (3,) and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/onnx_tf_prefix_/model.22/dfl/Constant with shape (4,) and dtype int64 was auto converted to the type int32',
 'weight StatefulPartitionedCall/Where_3 with shape (0, 1) and dtype

# Compress and download model

In [13]:
!tar vcfz {tfjs_model_dir}.tar.gz {tfjs_model_dir}


a ./runs/train/yolov8_hospital/weights/tf/hospital_web_model
a ./runs/train/yolov8_hospital/weights/tf/hospital_web_model/model.json
a ./runs/train/yolov8_hospital/weights/tf/hospital_web_model/group1-shard3of3.bin
a ./runs/train/yolov8_hospital/weights/tf/hospital_web_model/group1-shard1of3.bin
a ./runs/train/yolov8_hospital/weights/tf/hospital_web_model/group1-shard2of3.bin


In [14]:
file_path = tfjs_model_dir + '.tar.gz' # replace with your file path
destination_folder = './destination/'  # replace with your destination folder

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=destination_folder)
